# Advent of Code 2021 - Day 13 - Input Generator

This notebook contains a quickly crafted input generator for Advent of Code's Transparent Origami problem. Modify the output string and number of folds at the bottom and run the cell.

## Parse Character Set

The character set used is a monospaced 4x5 pixel font where the first line of input represents the character and the next 5 lines represents the specific pixels that make up the character.

In [33]:
enum class InputChar(val value: Char) {
    EMPTY('.'),
    PIXEL('█'),
}

enum class InputSize(val value: Int) {
    WIDTH(4),
    HEIGHT(5),
}

typealias Dot = Pair<Int, Int>

typealias Charset = Map<Char, Set<Dot>>

In [34]:
import java.io.File
import java.util.Scanner
import kotlin.sequences.generateSequence

val charset: Charset = Scanner(File("4x5.charset")).apply { useDelimiter("\n\n") }.use { scanner ->
    generateSequence { if (scanner.hasNext()) scanner.next() else null }
        .associate { entry ->
            entry.lines().let { lines ->
                lines.first().first() to lines.drop(1).flatMapIndexed { y, row ->
                    row.mapIndexedNotNull { x, input -> if (input == InputChar.PIXEL.value) x to y else null }
                }.toSet()
            }
        }
}

## Generate the Pixelated Message

Generate the dots for the final message which represents our solution.

In [35]:
fun String.pixelate(charset: Charset): Set<Dot> =
    flatMapIndexed { index, char ->
        charset[char]?.map { (x, y)  ->
            (x + (index * (InputSize.WIDTH.value + 1))) to y
        } ?: emptyList<Dot>()
    }.toSet()

## Unfold the Pixelated Message

Unfolds the pixelated message according to the instruction that would be used to fold it. Since the trick of the problem was to subtract the dots x/y position by 2 times the difference between the dot and the fold line, we can add to unfold it.

In [36]:
data class Instruction(val across: Direction, val at: Int)

enum class Direction {
    X,
    Y,
}

In [37]:
fun Dot.unfold(instruction: Instruction): Dot =
    when (instruction.across) {
        Direction.Y -> first to (instruction.at + abs(instruction.at - second))
        Direction.X -> (instruction.at + abs(instruction.at - first)) to second
    }

fun Set<Dot>.unfold(instruction: Instruction): Set<Dot> = map { it.unfold(instruction) }.toSet()

## Generate the Instructions

Generate the set of instructions to unfold the message that can be used to eventually refold the message.

In [38]:
import kotlin.random.Random

fun Set<Dot>.randomInstruction(): Pair<Set<Dot>, Instruction> =
    when (Random.nextBoolean()) {
        true -> Instruction(Direction.X, maxOf { it.first } + 1)
        false -> Instruction(Direction.Y, maxOf { it.second } + 1)
    }.let { instruction ->
        flatMap {
            val random = Random.nextInt(10)
            if (random < 3) listOf(it.unfold(instruction), it)
            else if (random < 7) listOf(it.unfold(instruction))
            else listOf(it)
        }.toSet() to instruction
    }

fun Set<Dot>.unfoldRandomly(times: Int = 1): Pair<Set<Dot>, List<Instruction>> {
    val instructions = mutableListOf<Instruction>()

    val dots = (1..times).fold(this) { acc, _ ->
        acc.randomInstruction()
            .also { instructions.add(it.second) }
            .first
    }

    return dots to instructions
}

## Serialize the Instructions

Generate the output as a set of instructions that can be parsed as input for the secret message.

In [39]:
fun Set<Dot>.toInput(): String = joinToString("\n") { "${it.first},${it.second}" }

fun List<Instruction>.toInput(): String = joinToString("\n") { "fold along ${it.across.name.lowercase()}=${it.at}" }

In [40]:
val message = "HELLO_WORLD"
val numberOfFolds = 15

message.pixelate(charset)
    .unfoldRandomly(times = numberOfFolds)
    .let { (dots, instructions) -> "${dots.toInput()}\n\n${instructions.reversed().toInput()}" }

878,3376
220,5572
658,10532
220,4740
658,9316
658,6262
658,156
658,11038
658,1180
220,10340
220,1878
220,7286
220,4932
220,11038
220,1180
658,8368
658,3850
658,6140
658,6078
658,34
220,3024
220,6078
878,10270
0,10270
0,1948
878,3402
878,5764
0,9508
550,8282
550,3936
550,11336
328,8282
328,5230
328,11336
550,2412
328,3700
550,4462
550,1650
328,8614
328,9710
328,2508
768,7226
768,11098
768,1120
768,6514
768,5704
110,8756
110,3462
110,5704
110,11810
110,408
548,4366
548,1746
548,12052
438,11290
438,928
438,6706
440,10856
658,6538
658,11786
658,432
0,10354
0,7300
878,8830
878,3388
878,9494
0,3388
0,334
545,4090
545,2022
545,11182
545,7910
545,8890
545,11374
333,8320
333,6950
773,6598
773,5620
105,6598
105,11726
773,10964
105,7360
105,6214
105,6004
105,12110
550,4329
550,7381
550,4837
768,823
110,8341
110,3877
110,9983
110,5407
770,10943
770,1275
108,1783
108,10943
108,1275
770,10051
770,6997
770,7765
770,10559
770,1659
770,7505
770,4713
108,10559
108,7505
108,10819
108,1399
328,11423
328,7